In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finlab.data import Data

pd.set_option('display.max_columns', None)

In [55]:
database = Data()

close = database.get("收盤價")
open_ = database.get("開盤價")
high = database.get("最高價")
low = database.get("最低價")
vol = database.get("成交股數")
rev = database.get("當月營收")
com_rev = database.get("上月比較增減(%)")
d_yield = database.get("殖利率(%)")
pb = database.get("股價淨值比")

sid = '2607'

data = pd.DataFrame({'close': close[sid],
                   'open': open_[sid],
                   'high': high[sid],
                   'low': low[sid],
                   'volume': vol[sid]})

data.tail()
print(f'shape of df {data.shape}')

shape of df (3621, 5)


In [56]:
data = data.reset_index()
data.dropna(axis=0, how='any', inplace=True)
rev = rev.reset_index()
print(f'shape of df {data.shape}')
data = pd.merge(data,rev[['date',sid]], on="date", how='outer')
print(f'shape of df {data.shape}')
data = data.sort_values(by=['date'])
data = data.rename(columns={sid: "rev"})
data['rev'].fillna(method='ffill', inplace=True)

com_rev = com_rev.reset_index()
print(f'shape of df {data.shape}')
data = pd.merge(data,com_rev[['date',sid]], on="date", how='outer')
print(f'shape of df {data.shape}')
data = data.sort_values(by=['date'])
data = data.rename(columns={sid: "com_rev"})
data['com_rev'].fillna(method='ffill', inplace=True)

d_yield = d_yield.reset_index()
print(f'shape of df {data.shape}')
data = pd.merge(data,d_yield[['date',sid]], on="date", how='outer')
print(f'shape of df {data.shape}')
data = data.rename(columns={sid: "d_yield"})
data['d_yield'].fillna(method='ffill', inplace=True)

pb = pb.reset_index()
print(f'shape of df {data.shape}')
data = pd.merge(data,pb[['date',sid]], on="date", how='outer')
print(f'shape of df {data.shape}')
data = data.rename(columns={sid: "pb"})
data['pb'].fillna(method='ffill', inplace=True)

data = data.set_index('date')
print(f'shape of df {data.shape}')
data.dropna(axis=0, how='any', inplace=True)
print(f'shape of df {data.shape}')
data.tail()

shape of df (3621, 6)
shape of df (3710, 7)
shape of df (3710, 7)
shape of df (3710, 8)
shape of df (3710, 8)
shape of df (3710, 9)
shape of df (3710, 9)
shape of df (3710, 10)
shape of df (3710, 9)
shape of df (2869, 9)


,close,open,high,low,volume,rev,com_rev,d_yield,pb
date,,,,,,,,,
2021-12-13,29.00,29.40,29.55,28.85,4471856.0,1032413.0,-6.86,1.03,1.17
2021-12-14,28.30,29.00,29.40,28.20,5130814.0,1032413.0,-6.86,1.06,1.14
2021-12-15,28.60,28.20,28.95,28.20,2777302.0,1032413.0,-6.86,1.05,1.16
2021-12-16,28.45,28.90,28.95,28.35,2274267.0,1032413.0,-6.86,1.05,1.15
2021-12-17,28.45,28.55,29.20,28.35,3365287.0,1032413.0,-6.86,1.05,1.15


In [57]:
def relative_strength_index(df, n):
        """Calculate Relative Strength Index(RSI) for given data.
        https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
        
        :param df: pandas.DataFrame
        :param n: 
        :return: pandas.DataFrame
        """
        i = 0
        UpI = [0]
        DoI = [0]
        while i + 1 <= df.index[-1]:
            UpMove = df.loc[i + 1, 'high'] - df.loc[i, 'high']
            DoMove = df.loc[i, 'low'] - df.loc[i + 1, 'low']
            if UpMove > DoMove and UpMove > 0:
                UpD = UpMove
            else:
                UpD = 0
            UpI.append(UpD)
            if DoMove > UpMove and DoMove > 0:
                DoD = DoMove
            else:
                DoD = 0
            DoI.append(DoD)
            i = i + 1
        UpI = pd.Series(UpI)
        DoI = pd.Series(DoI)
        PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
        NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
        RSI = pd.Series(round(PosDI * 100. / (PosDI + NegDI)), name='RSI_' + str(n))
        # df = df.join(RSI)
        return RSI

def get_rsi(data, window=14):
    df = data.copy(deep=True).reset_index()
    rsi = relative_strength_index(df, window)
    rsi_df = pd.Series(data=rsi.values, index=data.index)
    return rsi_df

def bbands(close_prices, window, no_of_stdev):
    # rolling_mean = close_prices.rolling(window=window).mean()
    # rolling_std = close_prices.rolling(window=window).std()
    rolling_mean = close_prices.ewm(span=window).mean()
    rolling_std = close_prices.ewm(span=window).std()

    upper_band = rolling_mean + (rolling_std * no_of_stdev)
    lower_band = rolling_mean - (rolling_std * no_of_stdev)

    return rolling_mean, upper_band, lower_band

def MA(close, n):
    return close.rolling(window=n).mean()

def bias(close, n):
    return close / close.rolling(n, min_periods=1).mean()

def acc(close, n):
    return close.shift(n) / (close.shift(2*n) + close) * 2

def mom(rev, n):
    return (rev / rev.shift(1)).shift(n)

In [58]:
data['MA5'] = MA(data['close'], 5)
data['MA60'] = MA(data['close'], 60)

data['bias5'] = bias(data['close'], 5)
data['bias10'] = bias(data['close'], 10)
data['bias20'] = bias(data['close'], 20)
data['bias60'] = bias(data['close'], 60)
# data['bias120'] = bias(data['close'], 120)
# data['bias240'] = bias(data['close'], 240)

data['acc5'] = acc(data['close'], 5)
data['acc10'] = acc(data['close'], 10)
data['acc20'] = acc(data['close'], 20)
data['acc60'] = acc(data['close'], 60)
# data['acc120'] = acc(data['close'], 120)
# data['acc240'] = acc(data['close'], 240)

# 用處不大，需在rev merge前就計算完成
# data['mom5'] = mom(data['rev'], 5)
# data['mom10'] = mom(data['rev'], 10)
# data['mom20'] = mom(data['rev'], 20)
# data['mom60'] = mom(data['rev'], 60)
# data['mom120'] = mom(data['rev'], 120)
# data['mom240'] = mom(data['rev'], 240)

In [59]:
window = 20
data['avg'], data['upper'], data['lower'] = bbands(data['close'], window, no_of_stdev=1.5)
rsi_df = get_rsi(data, window=14)
data['rsi'] = pd.Series(data=rsi_df.values, index=data.index)
# Compute sides
data['side'] = np.nan 
data['cs1'] = data['close'].shift(1)
data['ma60s3'] = data['MA60'].shift(3)
long_signals = (data['close'] >= data['MA5']) & (data['cs1'] <= data['MA5']) & (data['ma60s3'] <= data['MA60'])
short_signals = (data['close'] <= data['MA5']) & (data['cs1'] >= data['MA5']) & (data['ma60s3'] >= data['MA60']) 

data.loc[long_signals, 'side'] = 1
data.loc[short_signals, 'side'] = -1

print(data.side.value_counts())

# Remove Look ahead biase by lagging the signal
data['side'] = data['side'].shift(1)

 1.0    261
-1.0    250
Name: side, dtype: int64


In [60]:
data['side'].fillna(value=0, inplace=True)
raw_data = data.copy()
raw_data.tail()

,close,open,high,low,volume,rev,com_rev,d_yield,pb,MA5,MA60,bias5,bias10,bias20,bias60,acc5,acc10,acc20,acc60,avg,upper,lower,rsi,side,cs1,ma60s3
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-12-13,29.00,29.40,29.55,28.85,4471856.0,1032413.0,-6.86,1.03,1.17,29.77,28.725000,0.974135,0.978903,0.989423,1.009574,1.045217,0.998249,1.000890,1.036424,29.249908,30.513594,27.986223,53.0,-1.0,29.35,28.825000
2021-12-14,28.30,29.00,29.40,28.20,5130814.0,1032413.0,-6.86,1.06,1.14,29.40,28.685833,0.962585,0.958510,0.968018,0.986550,1.046875,1.008606,1.064286,0.976115,29.159441,30.435750,27.883132,43.0,0.0,29.00,28.795000
2021-12-15,28.60,28.20,28.95,28.20,2777302.0,1032413.0,-6.86,1.05,1.16,29.06,28.634167,0.984171,0.969820,0.979033,0.998807,1.052997,1.004337,1.026502,0.952667,29.106161,30.346200,27.866122,43.0,0.0,28.30,28.763333
2021-12-16,28.45,28.90,28.95,28.35,2274267.0,1032413.0,-6.86,1.05,1.15,28.74,28.601667,0.989910,0.968840,0.974983,0.994697,1.033534,1.032146,1.032831,0.866714,29.043669,30.259858,27.827480,43.0,0.0,28.60,28.725000
2021-12-17,28.45,28.55,29.20,28.35,3365287.0,1032413.0,-6.86,1.05,1.15,28.56,28.562500,0.996148,0.971985,0.974816,0.996061,1.014693,1.035211,1.028105,0.837525,28.987129,30.174637,27.799621,48.0,0.0,28.45,28.685833


In [61]:
# Log Returns
raw_data['log_ret'] = np.log(raw_data['close']).diff()

# Momentum
raw_data['mom1'] = raw_data['close'].pct_change(periods=1)
raw_data['mom2'] = raw_data['close'].pct_change(periods=2)
raw_data['mom3'] = raw_data['close'].pct_change(periods=3)
raw_data['mom4'] = raw_data['close'].pct_change(periods=4)
raw_data['mom5'] = raw_data['close'].pct_change(periods=5)

# Volatility
window_stdev = 50
raw_data['volatility'] = raw_data['log_ret'].rolling(window=window_stdev, min_periods=window_stdev, center=False).std()

# Serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_2'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=2), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)
raw_data['autocorr_4'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=4), raw=False)
raw_data['autocorr_5'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=5), raw=False)

# Get the various log -t returns
raw_data['log_t1'] = raw_data['log_ret'].shift(1)
raw_data['log_t2'] = raw_data['log_ret'].shift(2)
raw_data['log_t3'] = raw_data['log_ret'].shift(3)
raw_data['log_t4'] = raw_data['log_ret'].shift(4)
raw_data['log_t5'] = raw_data['log_ret'].shift(5)

# Add fast and slow moving averages
fast_window = 7
slow_window = 15

raw_data['fast_mavg'] = raw_data['close'].rolling(window=fast_window, min_periods=fast_window, center=False).mean()
raw_data['slow_mavg'] = raw_data['close'].rolling(window=slow_window, min_periods=slow_window, center=False).mean()


In [62]:
# Add Trending signals
raw_data['sma'] = np.nan

long_signals = raw_data['fast_mavg'] >= raw_data['slow_mavg']
short_signals = raw_data['fast_mavg'] < raw_data['slow_mavg']
raw_data.loc[long_signals, 'sma'] = 1
raw_data.loc[short_signals, 'sma'] = -1
raw_data['sma'].fillna(value=0, inplace=True)
raw_data.tail()

,close,open,high,low,volume,rev,com_rev,d_yield,pb,MA5,MA60,bias5,bias10,bias20,bias60,acc5,acc10,acc20,acc60,avg,upper,lower,rsi,side,cs1,ma60s3,log_ret,mom1,mom2,mom3,mom4,mom5,volatility,autocorr_1,autocorr_2,autocorr_3,autocorr_4,autocorr_5,log_t1,log_t2,log_t3,log_t4,log_t5,fast_mavg,slow_mavg,sma
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-12-13,29.00,29.40,29.55,28.85,4471856.0,1032413.0,-6.86,1.03,1.17,29.77,28.725000,0.974135,0.978903,0.989423,1.009574,1.045217,0.998249,1.000890,1.036424,29.249908,30.513594,27.986223,53.0,-1.0,29.35,28.825000,-0.011997,-0.011925,-0.034942,-0.042904,-0.038143,-0.034942,0.024431,-0.460299,0.058828,-0.020611,0.180534,-0.269220,-0.023570,-0.008285,0.004963,0.003322,0.021868,29.757143,29.366667,1.0
2021-12-14,28.30,29.00,29.40,28.20,5130814.0,1032413.0,-6.86,1.06,1.14,29.40,28.685833,0.962585,0.958510,0.968018,0.986550,1.046875,1.008606,1.064286,0.976115,29.159441,30.435750,27.883132,43.0,0.0,29.00,28.795000,-0.024434,-0.024138,-0.035775,-0.058236,-0.066007,-0.061360,0.023507,-0.408312,-0.000424,0.069557,0.164732,-0.379853,-0.011997,-0.023570,-0.008285,0.004963,0.003322,29.600000,29.336667,1.0
2021-12-15,28.60,28.20,28.95,28.20,2777302.0,1032413.0,-6.86,1.05,1.16,29.06,28.634167,0.984171,0.969820,0.979033,0.998807,1.052997,1.004337,1.026502,0.952667,29.106161,30.346200,27.866122,43.0,0.0,28.30,28.763333,0.010545,0.010601,-0.013793,-0.025554,-0.048253,-0.056106,0.023064,-0.381139,-0.056442,0.071060,0.222168,-0.359309,-0.024434,-0.011997,-0.023570,-0.008285,0.004963,29.392857,29.286667,1.0
2021-12-16,28.45,28.90,28.95,28.35,2274267.0,1032413.0,-6.86,1.05,1.15,28.74,28.601667,0.989910,0.968840,0.974983,0.994697,1.033534,1.032146,1.032831,0.866714,29.043669,30.259858,27.827480,43.0,0.0,28.60,28.725000,-0.005259,-0.005245,0.005300,-0.018966,-0.030664,-0.053245,0.022319,-0.342839,-0.067512,0.008649,0.196405,-0.320642,0.010545,-0.024434,-0.011997,-0.023570,-0.008285,29.150000,29.220000,-1.0
2021-12-17,28.45,28.55,29.20,28.35,3365287.0,1032413.0,-6.86,1.05,1.15,28.56,28.562500,0.996148,0.971985,0.974816,0.996061,1.014693,1.035211,1.028105,0.837525,28.987129,30.174637,27.799621,48.0,0.0,28.45,28.685833,0.000000,0.000000,-0.005245,0.005300,-0.018966,-0.030664,0.021680,-0.339244,-0.008307,0.046043,0.157159,-0.326820,-0.005259,0.010545,-0.024434,-0.011997,-0.023570,28.885714,29.236667,-1.0


In [63]:
raw_data = raw_data.shift(1)
raw_data.dropna(axis=0, how='any', inplace=True)

In [64]:
# Get features at event dates
raw_data = raw_data.reset_index()
X = raw_data.copy()

# Drop unwanted columns
X.drop(['date', 'open', 'high', 'low','volume','rev','avg', 'upper', 'lower', 'close','cs1', 'ma60s3', 'sma','fast_mavg', 'slow_mavg'], axis=1, inplace=True)

# y = raw_data['out']
X.head()

,com_rev,d_yield,pb,MA5,MA60,bias5,bias10,bias20,bias60,acc5,acc10,acc20,acc60,rsi,side,log_ret,mom1,mom2,mom3,mom4,mom5,volatility,autocorr_1,autocorr_2,autocorr_3,autocorr_4,autocorr_5,log_t1,log_t2,log_t3,log_t4,log_t5
0,-2.82,1.90,1.36,26.71,25.696667,0.986522,0.976649,1.005821,1.025425,0.996255,1.045411,0.969466,0.950287,53.0,0.0,-0.016934,-0.016791,-0.018622,-0.022263,-0.009398,-0.009398,0.020711,-0.154889,0.041217,-0.209722,0.155046,0.119823,-0.001864,-0.003717,0.013072,0.000000,-0.025975
1,-2.82,1.92,1.35,26.61,25.719167,0.980834,0.968999,0.994665,1.014807,1.010446,1.034080,0.952830,0.950096,46.0,0.0,-0.009533,-0.009488,-0.026119,-0.027933,-0.031540,-0.018797,0.020452,-0.182935,0.046877,-0.195303,0.139083,0.118809,-0.016934,-0.001864,-0.003717,0.013072,0.000000
2,-2.82,1.90,1.36,26.48,25.748333,0.993202,0.983545,0.999810,1.021425,0.988084,1.101365,0.945180,0.946050,44.0,0.0,0.007634,0.007663,-0.001898,-0.018657,-0.020484,-0.024119,0.020127,-0.185008,0.057761,-0.212715,0.136489,0.124918,-0.009533,-0.016934,-0.001864,-0.003717,0.013072
3,-2.82,1.85,1.40,26.53,25.785833,1.021485,1.015171,1.025835,1.050965,0.982617,1.060635,0.954851,0.952107,59.0,0.0,0.029965,0.030418,0.038314,0.028463,0.011194,0.009311,0.020574,-0.171544,0.044586,-0.231226,0.130722,0.116849,0.007634,-0.009533,-0.016934,-0.001864,-0.003717
4,-2.82,1.87,1.39,26.53,25.814167,1.010177,1.005817,1.010558,1.038190,0.990758,1.059166,0.962099,0.989163,63.0,1.0,-0.011132,-0.011070,0.019011,0.026820,0.017078,0.000000,0.020549,-0.179060,0.041168,-0.225904,0.140572,0.099921,0.029965,0.007634,-0.009533,-0.016934,-0.001864


In [65]:
from sklearn.externals import joblib

In [66]:
rf = joblib.load('0209_rf.pkl')

In [67]:
y_pred_rf = rf.predict_proba(X)[:, 1]

In [68]:
y_pred_rf.shape

(2748,)

In [77]:
fdf = raw_data.copy()
fdf = fdf[['date', 'close', 'open', 'high', 'low','volume']]
fdf = fdf.rename(columns={"date": "datetime"})
fdf['openinterest'] = y_pred_rf
fdf.head()

,datetime,close,open,high,low,volume,openinterest
0,2010-10-19,26.35,27.0,27.05,26.30,3659159.0,0.584795
1,2010-10-20,26.10,26.4,26.65,26.00,2343239.0,0.625937
2,2010-10-21,26.30,26.0,26.50,25.90,2505356.0,0.395891
3,2010-10-22,27.10,26.3,27.20,26.25,5585503.0,0.425043
4,2010-10-25,26.80,27.1,27.40,26.80,4753909.0,0.470111


In [78]:
fdf.to_csv('test.csv', index=False)